In [0]:
pathRaw = "/Volumes/raw/astronauts/list_astronauts"
catalog = "bronze"
schema = "astronauts"
tableName = dbutils.widgets.get("tableName")

In [0]:
df_full = (spark.read.
            format("csv").
            load(f"{pathRaw}", 
                 header=True, 
                 inferSchema=True
            )
        )

In [0]:
def table_exists(catalog, schema, table):
    count = (spark.sql(f"SHOW TABLES FROM {catalog}.{schema} WHERE tableName = '{table}')
                .count()
            )
    return count > 0

In [0]:
import delta

if not table_exists(f"{catalog}.{schema}.{tableName}"):
    
    print("Table is creating now...")
    (df_full
        .coalesce(1).
        write.
        format("delta").
        mode("overwrite").
        saveAsTable(f"{catalog}.{schema}.{tableName}")
    )
else:
    deltaTable = DeltaTable.forName(f"{catalog}.{schema}.{tableName}")
    deltaTable.alias("t").merge(
        df_full.alias("s"),
        f"t.id = s.id"
    ).whenNotMatchedInsertAll().execute()